In [1]:
# При запуске ноутбука рабочей директорией ставится не папка проекта, а папка examples, а в ней нет src, 
# соответственно импорты не работают. Эта строчка фиксит такое поведение для тестирования, у пользователей будет
# свой путь установки, в котором этой проблемы не будет. 
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


FileNotFoundError: [Errno 2] No such file or directory: 'simple-onlineshop.csv'

In [7]:
# Add simple group
from src.data_processors_lib.simple_processors import SimpleGroup, SimpleGroupParams

cart_events = EventsNode(
    SimpleGroup(params=SimpleGroupParams(**{
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(["cart"]),
    }))
)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)

In [8]:
from src.data_processors_lib.rete import StartEndEvents, StartEndEventsParams

start_end_events = EventsNode(
    StartEndEvents(params=StartEndEventsParams(**{}))
)

graph.add_node(
    node=start_end_events,
    parents=[cart_events]
)

result = graph.combine(
    node=start_end_events
)
result.to_dataframe().head(n=20)


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,05e3d74c-875b-43ac-8934-5a0640f299fb,start,0,start,2019-11-01 17:59:13.273932,219483890.0
2,ebfec27d-3ca2-4a6f-821f-3ce75ed97252,raw,2,product1,2019-11-01 17:59:28.459271,219483890.0
3,a0bcbff7-28cd-437d-aa75-b5de99c1460c,group_alias,3,add_to_cart,2019-11-01 17:59:29.502214,219483890.0
5,09d869fa-357f-49e3-b2ce-decd8bde77ef,raw,5,catalog,2019-11-01 17:59:32.557029,219483890.0
6,de326d44-6f9e-449f-b536-3b72e58a7f2f,start,6,start,2019-11-01 21:38:19.283663,964964743.0
8,6674e956-25ff-4065-b538-45a55873f06b,group_alias,8,add_to_cart,2019-11-01 21:38:36.761221,964964743.0
10,9c2b5a57-2985-46f6-a8cf-b2e6891cd6a7,raw,10,delivery_choice,2019-11-01 21:38:37.564693,964964743.0
11,1cf55233-7d23-4805-80ec-ee9756df5ce0,start,11,start,2019-11-01 22:28:54.791683,629881394.0
12,decd4149-b621-456e-81b0-cb2a99c52815,start,12,start,2019-11-01 22:28:54.791683,629881395.0
15,35922af0-c921-4fa2-b37c-1cc7fb768cf6,raw,15,catalog,2019-11-01 22:29:01.049513,629881394.0
